## Data Preparation
- The purpose is to learn about data preparation using pandas in order to convert data into formats that is eaiser to gain insights from

### Import libraries

In [1]:
import pandas as pd

### Load data

In [2]:
superstore = pd.read_excel('/Users/apple/Downloads/big-data/Lecture 2/M2_Superstore.xlsx', index_col='Row ID')

superstore.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2013-152156,2014-11-09,2014-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2013-152156,2014-11-09,2014-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,CA-2013-138688,2014-06-13,2014-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
4,US-2012-108966,2013-10-11,2013-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2012-108966,2013-10-11,2013-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Check data types

In [3]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 1 to 9994
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9994 non-null   object        
 1   Order Date     9994 non-null   datetime64[ns]
 2   Ship Date      9994 non-null   datetime64[ns]
 3   Ship Mode      9994 non-null   object        
 4   Customer ID    9994 non-null   object        
 5   Customer Name  9994 non-null   object        
 6   Segment        9994 non-null   object        
 7   Country        9994 non-null   object        
 8   City           9994 non-null   object        
 9   State          9994 non-null   object        
 10  Postal Code    9994 non-null   int64         
 11  Region         9994 non-null   object        
 12  Product ID     9994 non-null   object        
 13  Category       9994 non-null   object        
 14  Sub-Category   9994 non-null   object        
 15  Product Name   9994 n

### Recency-Frequency-Monetary (RFM)

In [4]:
currentDate = superstore['Order Date'].max()

In [5]:
def recency(orderDates):
    return (currentDate - orderDates.max()).days

In [6]:
rfmData = (
    superstore
    .groupby('Customer ID')
    .agg({
    'Order Date': recency, # time elapsed (days) since last order
    'Order ID' : 'nunique', # count of order
    'Sales': 'sum' # customer's total spending with store
    })
    .rename(columns={'Order Date': 'R', 'Order ID': 'F', 'Sales': 'M'})
)

rfmData.head()

,R,F,M
Customer ID,,,
AA-10315,184,5,5563.560
AA-10375,19,9,1056.390
AA-10480,259,4,1790.512
AA-10645,55,6,5086.935
AB-10015,415,3,886.156


In [7]:
# get which quartile of the column each value is in
rfmData.transform(lambda x: pd.qcut(x, q=4, labels=False))

,R,F,M
Customer ID,,,
AA-10315,3,0,3
AA-10375,0,3,0
AA-10480,3,0,1
AA-10645,1,1,3
AB-10015,3,0,0
...,...,...,...
XP-21865,1,3,2
YC-21895,0,0,3
YS-21880,0,2,3


### Product monthly sales trend as crosstabulation

In [8]:
monthlySales = (
    superstore
    .set_index('Order Date') # for resample
    .groupby(['Category', 'Sub-Category'])
    .resample('M') # bin into months and get upper bound (last date of month)
    .agg({
        'Sales': 'sum'
    })
    .reset_index()
    .pivot_table(
        index = ['Category', 'Sub-Category'],
        columns = 'Order Date',
        values = 'Sales',
        aggfunc = 'sum',
        fill_value = 0
    )
)
monthlySales

Order Date                    2012-01-31  2012-02-29  2012-03-31  2012-04-30  \
Category        Sub-Category                                                   
Furniture       Bookcases        719.389     290.666    1706.446     308.499   
                Chairs          4187.700     190.458    5713.142    3210.666   
                Furnishings      711.770     392.980     480.128     984.046   
                Tables           333.000    1256.220    6674.240    3441.626   
Office Supplies Appliances       312.580      89.920     502.962     532.580   
                Art              176.990      73.664     413.268     567.916   
                Binders          814.508     339.262    1525.675     985.752   
                Envelopes        194.360       0.000     353.200     408.352   
                Fasteners         37.060      12.350      48.240     101.684   
                Labels            44.746      36.876     173.590      78.992   
                Paper            242.020     138.488    1200.322     625.432   
                Storage         3028.816     142.544    4370.002    3680.012   
                Supplies           0.000     238.620      18.620    4174.354   
Technology      Accessories      827.890    1120.990    1108.934    2545.990   
                Copiers            0.000       0.000       0.000       0.000   
                Machines           0.000       0.000   25314.658     287.968   
                Phones          2315.400     487.520    6087.582    6361.476   

Order Date                    2012-05-31  2012-06-30  2012-07-31  2012-08-31  \
Category        Sub-Category                                                   
Furniture       Bookcases        640.631    759.5736    1487.673    794.2760   
                Chairs          3675.109   6456.0010    6364.940   2590.2420   
                Furnishings      780.166    432.7500     902.622    997.7580   
                Tables          1816.881   5557.8010    2065.816   2938.0705   
Office Supplies Appliances       918.596   2275.2460      81.740   2096.7530   
                Art              288.220    686.1120     256.040    203.6620   
                Binders         4372.304   4275.7930    2934.492   4251.3870   
                Envelopes        214.194     95.5600     136.864    203.3320   
                Fasteners         12.576     45.2920      24.348     85.8700   
                Labels            57.440    607.7260     194.790     41.3760   
                Paper            479.112   1090.1900     796.332   2017.3840   
                Storage          712.902   3849.5140    2452.732   2322.7300   
                Supplies          80.280     27.6040    8243.870    156.9680   
Technology      Accessories      695.490    667.0400    3308.692   1937.7340   
                Copiers         2999.930      0.0000     559.992   1749.9660   
                Machines           0.000    691.7610       0.000      0.0000   
                Phones          5904.456   7077.1640    4135.450   5521.9600   

Order Date                    2012-09-30  2012-10-31  ...  2015-03-31  \
Category        Sub-Category                          ...               
Furniture       Bookcases      2394.4698     616.998  ...    1722.860   
                Chairs        13849.1850    6486.605  ...    5524.914   
                Furnishings    2960.9560     621.812  ...     872.454   
                Tables         4611.8700    4578.832  ...    2477.104   
Office Supplies Appliances     2413.2640     430.792  ...    2439.994   
                Art             759.8600     553.526  ...     442.084   
                Binders       12743.6160     480.156  ...    2683.200   
                Envelopes       381.3720     340.512  ...     517.016   
                Fasteners        95.4780      51.988  ...      24.680   
                Labels          381.5360      99.102  ...     224.100   
                Paper          1445.3580     990.846  ...    2483.840   
         

### User transaction matrix

In [9]:
userTransMat = (
    superstore
    .pivot_table(
        index = 'Customer ID',
        columns =  ['Category', 'Sub-Category'],
        values = 'Order ID',
        aggfunc = 'nunique',
        fill_value = 0
    )
)
userTransMat

Category     Furniture                           Office Supplies              \
Sub-Category Bookcases Chairs Furnishings Tables      Appliances Art Binders   
Customer ID                                                                    
AA-10315             0      0           1      0               2   0       2   
AA-10375             0      0           1      0               0   1       4   
AA-10480             0      0           1      1               0   1       0   
AA-10645             1      2           3      0               0   1       1   
AB-10015             1      1           0      0               0   1       0   
...                ...    ...         ...    ...             ...  ..     ...   
XP-21865             0      0           3      0               2   4       2   
YC-21895             0      0           1      0               0   1       0   
YS-21880             0      1           1      1               0   1       2   
ZC-21910             0      3           3      2               2   3       4   
ZD-21925             0      0           2      0               1   1       1   

Category                                                        Technology  \
Sub-Category Envelopes Fasteners Labels Paper Storage Supplies Accessories   
Customer ID                                                                  
AA-10315             0         1      0     1       1        1           1   
AA-10375             0         0      0     3       2        0           2   
AA-10480             0         0      0     3       1        0           1   
AA-10645             1         0      0     2       3        0           0   
AB-10015             0         0      0     0       2        0           0   
...                ...       ...    ...   ...     ...      ...         ...   
XP-21865             1         1      0     4       1        0           3   
YC-21895             0         0      0     2       2        0           0   
YS-21880             1         0      0     0       4        0           1   
ZC-21910             2         1      0     2       1        0           4   
ZD-21925             0         0      0     1       0        0           0   

Category                              
Sub-Category Copiers Machines Phones  
Customer ID                           
AA-10315           0        0      1  
AA-10375           0        0      1  
AA-10480           0        0      1  
AA-10645           0        0      2  
AB-10015           0        0      1  
...              ...      ...    ...  
XP-21865           0        0      0  
YC-21895           1        0      1  
YS-21880           0        0      0  
ZC-21910           0        0      2  
ZD-21925           0        0      3  

[793 rows x 17 columns]

### Customer behavior profiles

In [10]:
# check if an order is associated with multiple order dates
(
    superstore
    .groupby('Order ID')['Order Date']
    .nunique()
    .sort_values()
    .tail()
)

Order ID
CA-2012-166975    1
CA-2012-166947    1
CA-2012-166800    1
CA-2012-167479    1
US-2014-169551    1
Name: Order Date, dtype: int64

In [11]:
max_sales_order = (
    superstore
    .groupby(['Customer ID', 'Order ID'], as_index=False)
    .agg({
        'Sales': 'sum'
    })
    .set_index('Order ID')
    .groupby('Customer ID', as_index=False)
    .agg({
        'Sales': 'idxmax'
    })
    .rename(columns={
        'Sales': 'Order ID'
    })
    
)
max_sales_order['Max Sales'] = 1 # mark row as max sales order
max_sales_order.head()

,Customer ID,Order ID,Max Sales
0,AA-10315,CA-2013-103982,1
1,AA-10375,CA-2013-131065,1
2,AA-10480,CA-2013-114601,1
3,AA-10645,CA-2012-110863,1
4,AB-10015,CA-2013-140935,1


In [12]:
orderedRecord = (
    superstore[['Customer ID', 'Order ID', 'Order Date']]
    .drop_duplicates()
    .sort_values(['Customer ID', 'Order Date'])
)
orderedRecord.head()

,Customer ID,Order ID,Order Date
Row ID,,,
2230,AA-10315,CA-2011-128055,2012-03-31
7469,AA-10315,CA-2011-138100,2012-09-15
1300,AA-10315,CA-2012-121391,2013-10-04
5199,AA-10315,CA-2013-103982,2014-03-04
1160,AA-10315,CA-2014-147039,2015-06-30


In [13]:
orderedRecord_1 = orderedRecord.merge(max_sales_order, on=['Customer ID', 'Order ID'], how = 'left')
# move max sales flag of groupped rows 1,2,3 step up to mark up to past 3 orders
orderedRecord_1['M1'] = orderedRecord_1.groupby('Customer ID')['Max Sales'].shift(-1)
orderedRecord_1['M2'] = orderedRecord_1.groupby('Customer ID')['M1'].shift(-1)
orderedRecord_1['M3'] = orderedRecord_1.groupby('Customer ID')['M2'].shift(-1)
orderedRecord_1 = orderedRecord_1.fillna(0)
# mark row whether it is within past 3 orders from customer max sales order
orderedRecord_1['Past Records'] = orderedRecord_1['M1'] + orderedRecord_1['M2'] + orderedRecord_1['M3']
orderedRecord_1

,Customer ID,Order ID,Order Date,Max Sales,M1,M2,M3,Past Records
0,AA-10315,CA-2011-128055,2012-03-31,0.0,0.0,0.0,1.0,1.0
1,AA-10315,CA-2011-138100,2012-09-15,0.0,0.0,1.0,0.0,1.0
2,AA-10315,CA-2012-121391,2013-10-04,0.0,1.0,0.0,0.0,1.0
3,AA-10315,CA-2013-103982,2014-03-04,1.0,0.0,0.0,0.0,0.0
4,AA-10315,CA-2014-147039,2015-06-30,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5004,ZD-21925,CA-2011-143336,2012-08-27,0.0,0.0,0.0,1.0,1.0
5005,ZD-21925,CA-2013-167682,2014-04-04,0.0,0.0,1.0,0.0,1.0
5006,ZD-21925,US-2013-147991,2014-05-06,0.0,1.0,0.0,0.0,1.0
5007,ZD-21925,CA-2013-152471,2014-07-09,1.0,0.0,0.0,0.0,0.0


In [14]:
orderedRecord_2 = (
    orderedRecord_1.groupby('Customer ID', as_index=False)
    .agg({
        'Past Records': 'sum' # count no. of past orders
    })
)
orderedRecord_2.head(6)

,Customer ID,Past Records
0,AA-10315,3.0
1,AA-10375,3.0
2,AA-10480,2.0
3,AA-10645,2.0
4,AB-10015,2.0
5,AB-10060,3.0


In [15]:
# select ID of customers with at least 3 orders before max sales order
selectedCustomers = orderedRecord_2[orderedRecord_2['Past Records'] == 3].drop(columns=['Past Records'])
selectedCustomers.head(5)

,Customer ID
0,AA-10315
1,AA-10375
5,AB-10060
6,AB-10105
8,AB-10165


In [16]:
# select customers' past 3 orders before max sales order
orderedRecord_3 =(
    orderedRecord_1[orderedRecord_1['Past Records'] == 1]
    .merge(selectedCustomers, on=['Customer ID'])
    .drop(columns=['Max Sales', 'Past Records'])    
)
orderedRecord_3.head()

,Customer ID,Order ID,Order Date,M1,M2,M3
0,AA-10315,CA-2011-128055,2012-03-31,0.0,0.0,1.0
1,AA-10315,CA-2011-138100,2012-09-15,0.0,1.0,0.0
2,AA-10315,CA-2012-121391,2013-10-04,1.0,0.0,0.0
3,AA-10375,CA-2012-109939,2013-05-08,0.0,0.0,1.0
4,AA-10375,CA-2012-114503,2013-11-13,0.0,1.0,0.0


In [17]:
# combine flag columns into one column
orderedRecord_past = pd.melt(
    orderedRecord_3,
    id_vars=['Customer ID', 'Order ID', 'Order Date'],
    value_vars=['M1', 'M2', 'M3'],
    var_name='M',
    value_name='Value'
)

orderedRecord_past = (
    orderedRecord_past[orderedRecord_past['Value'] > 0]
    .drop(columns=['Value'])
    .sort_values(['Customer ID', 'Order Date']) # melting was done column by column, row order was messed up
)

orderedRecord_past.head()

,Customer ID,Order ID,Order Date,M
2154,AA-10315,CA-2011-128055,2012-03-31,M3
1078,AA-10315,CA-2011-138100,2012-09-15,M2
2,AA-10315,CA-2012-121391,2013-10-04,M1
2157,AA-10375,CA-2012-109939,2013-05-08,M3
1081,AA-10375,CA-2012-114503,2013-11-13,M2


In [18]:
sales_profit = (
    superstore
    .groupby(['Customer ID', 'Order ID'], as_index=False)
    .agg({
        'Sales': 'sum', # sales from order
        'Profit': 'sum' # order profit
    })
)
sales_profit.head()

,Customer ID,Order ID,Sales,Profit
0,AA-10315,CA-2011-128055,726.548,267.4224
1,AA-10315,CA-2011-138100,29.500,13.2826
2,AA-10315,CA-2012-121391,26.960,7.0096
3,AA-10315,CA-2013-103982,4406.072,-747.1021
4,AA-10315,CA-2014-147039,374.480,96.5050


In [19]:
ordered_final = (
    orderedRecord_past
    .merge(sales_profit, on=['Customer ID', 'Order ID'], how='inner')
    .pivot_table(
        index=['Customer ID'],
        columns=['M'],
        values=['Sales', 'Profit'],
        aggfunc='sum'
    )
)
ordered_final

Profit                        Sales                  
M                  M1        M2        M3       M1       M2       M3
Customer ID                                                         
AA-10315       7.0096   13.2826  267.4224   26.960   29.500  726.548
AA-10375       1.4672    6.3720    0.5904   16.768   84.960    5.248
AB-10060       8.0997  106.5112    8.7138   27.930  334.200   18.540
AB-10105    -204.4458    5.8980 -142.6601  393.165  117.960  671.002
AB-10165      -2.9180   40.9680   -1.0656   46.688  199.640   37.296
...               ...       ...       ...      ...      ...      ...
VP-21760      31.6459 -116.8440  -87.3009  198.670  166.920  366.666
WB-21850     -32.5272   53.7944 -108.9542   30.392  126.176  258.495
YS-21880       1.5522   15.5250  -38.2116    9.552   34.500  436.704
ZC-21910     -79.0382  -27.7534  -50.6688  349.082  910.792   38.976
ZD-21925       3.3440  146.8280   25.8774   16.720  331.080  244.760

[359 rows x 6 columns]